# BIK jaargang toevoegen tbv Power BI rapportage BIK

\- zie: C:\\Users\\jvdw\\Staedion\\INFO VZ T&C - BRONBESTANDEN PBIX\\BIK 2019-2022.pbix

## Achtergrondinformatie

Handmatige acties om PBI voor nieuwe jaargang BIK geschikt te maken

1. Aantallen woningen ultimo jaar toevoegen (empire\_staedion\_Data.bik.AantalWoningenPerBuurtUltimo + AantalWoningenPerClusterUltimo)
2. KCM thuisgevoel met aantal indicatoren toevoegen en verwerken in onderliggende views (staedion\_dm.Leefbaarheid.KlantContactMonitorOpCluster + KlantContactMonitorOpBuurt)
3. Check of overlastdossiers nieuwe jaargang automatisch opgehaald wordt (staedion\_dm.Leefbaarheid.LeefbaarheidsdossiersOpCluster + Leefbaarheid.LeefbaarheidsdossiersOpBuurt)
4. Teamscores excel van medewerkers toevoegen en verwerken in view (empire\_Staedion\_data.bik.TeamscoreJJJJ + staedion\_dm.Leefbaarheid.TeamscoreOpCluster +TeamscoreOpBuurt)

## 1. Bijwerken ELS-lijst-aantallen einde jaar voor cluster en buurt

In [ ]:
/* 
-- Aanvullen ELS_AantalWoningenPerClusterUltimo
INSERT INTO empire_staedion_data.bik.ELS_AantalWoningenPerClusterUltimo (Clusternummer, Clusternaam, Jaar, Aantalwoningen)
SELECT clusternummer, clusternaam, 2021,count(*) as Aantalwoningen
from empire_Staedion_data.dbo.els
where datum_gegenereerd = (select max(datum_gegenereerd) from  empire_Staedion_data.dbo.els where YEAR(datum_gegenereerd) = 2021) 
and Corpodata_type LIKE '%WON%'
AND datum_in_exploitatie <= '20211231' AND ( NULLIF(datum_uit_exploitatie,'') IS NULL OR datum_uit_exploitatie > '20211231') 
group by clusternummer, clusternaam
;
-- Aanvullen ELS_AantalWoningenPerBuurtUltimo
insert into empire_staedion_data.bik.ELS_AantalWoningenPerBuurtUltimo (Buurtcode, Jaar, AantalWoningen, AantalClusters)
SELECT REPLACE(REPLACE(E.fk_cbsbuurt_id, 'BU0',''),'BU1','')  as Buurtcode, 2021,count(*) as Aantalwoningen, COUNT(DISTINCT E.staedion_fk_ftcluster_id)
-- select top 10 E.*
from empire_dwh.dbo.eenheid as E
join empire_dwh.dbo.technischtype as T
on T.id = E.fk_technischtype_id
where T.fk_eenheid_type_corpodata_id LIKE '%WON%'
AND E.da_bedrijf = 'Staedion'
AND dt_in_exploitatie <= '20211231' AND ( NULLIF(dt_uit_exploitatie,'') IS NULL OR dt_uit_exploitatie > '20211231') 
group by REPLACE(REPLACE(E.fk_cbsbuurt_id, 'BU0',''),'BU1','')
;
*/

In [2]:
-- Check
select jaar, sum(Aantalwoningen) AS Aantalwoningen, count(clusternummer) AS Aantalclusters
from empire_staedion_data.bik.ELS_AantalWoningenPerClusterUltimo
group by jaar
order by jaar
;


Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:00.016

jaar,Aantalwoningen,Aantalclusters
2012,NULL,639
2013,NULL,639
2014,NULL,639
2015,NULL,639
2016,35703,639
2017,35703,639
2018,35764,641
2019,35829,645
2020,36921,683
2021,36913,680


In [4]:
-- Check
select jaar, sum(Aantalwoningen) AS Aantalwoningen, count(Buurtcode) AS Aantalbuurten
from empire_staedion_data.bik.ELS_AantalWoningenPerBuurtUltimo
group by jaar
order by jaar desc
;

Warning: Null value is eliminated by an aggregate or other SET operation.

(10 rows affected)

Total execution time: 00:00:00.007

jaar,Aantalwoningen,Aantalbuurten
2012,NULL,83
2013,NULL,83
2014,NULL,83
2015,NULL,83
2016,35703,83
2017,35703,83
2018,35764,82
2019,35829,82
2020,36921,90
2021,36945,90


## 2\. Aanvullen staedion\_dm.Leefbaarheid.KlantContactMonitorOpCluster + \[Leefbaarheid\].\[KlantContactMonitorOpBuurt\]

In [ ]:
/* 
-- ingelezen
SELECT Clusternr,
		YEAR(CAST([INGEVULDE GEGEVENS] AS DATE)) AS Jaar,
       COUNT(*) AS AantalRespondenten,
       AVG(CAST(NULLIF([Welk rapportcijfer geeft u voor uw 'thuisgevoel'? Een 1 staat hi],'') AS DECIMAL(8,2))) AS KCMThuisgevoel,
       AVG(CAST(NULLIF([Welk rapportcijfer geeft u voor de kwaliteit van uw woning? Een ],'') AS DECIMAL(8,2))) AS KCMTevredenheidWoning,
       AVG(CAST(NULLIF([Welk rapportcijfer geeft u Staedion voor de algemene ruimten ron],'') AS DECIMAL(8,2))) AS KCMTevredenheidAlgemeneRuimte,
       AVG(CAST(NULLIF([Welk rapportcijfer geeft u voor uw buurt? Een 1 staat hier voor ],'') AS DECIMAL(8,2))) AS KCMTevredenheidBuurt,

		AVG(CAST(NULLIF([Welk rapportcijfer geeft u voor uw 'thuisgevoel'? Een 1 staat hi],'') AS DECIMAL(8,2)))*0.25+
		AVG(CAST(NULLIF([Welk rapportcijfer geeft u voor de kwaliteit van uw woning? Een ],'') AS DECIMAL(8,2)))*0.25+
		AVG(CAST(NULLIF([Welk rapportcijfer geeft u Staedion voor de algemene ruimten ron],'') AS DECIMAL(8,2)))*0.25+
		AVG(CAST(NULLIF([Welk rapportcijfer geeft u voor uw buurt? Een 1 staat hier voor ],'') AS DECIMAL(8,2)))*0.25 AS KCMCijfer
INTO  [empire_staedion_data].[bik].[STN661_Ingevulde_gegevens_2021] 
FROM empire_staedion_data.kcm.stn661_ingevulde_gegevens
WHERE YEAR(CAST([INGEVULDE GEGEVENS] AS DATE)) = 2021
GROUP BY Clusternr,YEAR(CAST([INGEVULDE GEGEVENS] AS DATE))
;
*/

In [5]:
-- totalen
SELECT Jaar, SUM(AantalRespondenten),SUM(AantalRespondenten*KCMThuisgevoel)/SUM(AantalRespondenten)  as KCMThuisgevoel
		, SUM(AantalRespondenten*KCMTevredenheidWoning)/SUM(AantalRespondenten)  as KCMTevredenheidWoning
		, SUM(AantalRespondenten*KCMTevredenheidAlgemeneRuimte)/SUM(AantalRespondenten)  as KCMTevredenheidAlgemeneRuimte
		, SUM(AantalRespondenten*KCMTevredenheidBuurt)/SUM(AantalRespondenten)  as KCMTevredenheidBuurt
		, SUM(AantalRespondenten*KCMCijfer)/SUM(AantalRespondenten)  as KCMCijfer
FROM	staedion_dm.Leefbaarheid.[KlantContactMonitorOpCluster]
GROUP BY Jaar 
ORDER BY Jaar desc
;


Warning: Null value is eliminated by an aggregate or other SET operation.

(4 rows affected)

Total execution time: 00:00:00.627

Jaar,(No column name),KCMThuisgevoel,KCMTevredenheidWoning,KCMTevredenheidAlgemeneRuimte,KCMTevredenheidBuurt,KCMCijfer
2018,2613,"7,918101798698814","7,151549942594719","6,745909199558307","7,090317642556449","7,228000453353029"
2019,3831,"7,891412163925868","7,184025058731402","6,746177414553127","7,188201513965022","7,258588206418236"
2020,7691,"7,832921596671434","7,087504875828891","6,574679828534149","7,086594721102587","7,156390452951157"
2021,8004,"7,8255868526986365","7,07358786744126","6,530950993003494","7,225511880684639","7,179734818882224"


In [6]:
-- aanpassen view [KlantContactMonitorOpBuurt]
SELECT Jaar, SUM(AantalRespondenten),SUM(AantalRespondenten*KCMThuisgevoel)/SUM(AantalRespondenten)  as KCMThuisgevoel
		, SUM(AantalRespondenten*KCMTevredenheidWoning)/SUM(AantalRespondenten)  as KCMTevredenheidWoning
		, SUM(AantalRespondenten*KCMTevredenheidAlgemeneRuimte)/SUM(AantalRespondenten)  as KCMTevredenheidAlgemeneRuimte
		, SUM(AantalRespondenten*KCMTevredenheidBuurt)/SUM(AantalRespondenten)  as KCMTevredenheidBuurt
		, SUM(AantalRespondenten*KCMCijfer)/SUM(AantalRespondenten)  as KCMCijfer
FROM	staedion_dm.Leefbaarheid.[KlantContactMonitorOpBuurt]
GROUP BY Jaar 
ORDER BY Jaar desc
;

Warning: Null value is eliminated by an aggregate or other SET operation.

(4 rows affected)

Total execution time: 00:00:00.388

Jaar,(No column name),KCMThuisgevoel,KCMTevredenheidWoning,KCMTevredenheidAlgemeneRuimte,KCMTevredenheidBuurt,KCMCijfer
2018,3737,"7,895638212469896","7,123093390420123","6,723554319347293","7,075996788868076","7,2045706777763465"
2019,4765,"7,887513116474292","7,17292759706191","6,768867342967464","7,140608604407135","7,242479165227701"
2020,8001,"7,826021747281589","7,084989376327959","6,6186945905887455","7,0666166729158855","7,149080596778549"
2021,8346,"7,829486218308172","7,06510932446681","6,588137387371198","7,223397092020125","7,176532505541576"


## 3\. Check nieuwe jaargang staedion\_dm.Leefbaarheid.LeefbaarheidsdossiersOpBuurt + \[Leefbaarheid\].\[LeefbaarheidsdossiersOpCluster\]

In [7]:
SELECT Jaar, COUNT(*) FROM staedion_dm.Leefbaarheid.LeefbaarheidsdossiersOpCluster GROUP BY Jaar ORDER BY Jaar desc


(6 rows affected)

Total execution time: 00:00:00.022

Jaar,(No column name)
2021,680
2020,683
2019,645
2018,641
2017,639
2016,639


In [8]:
SELECT Jaar, COUNT(*) FROM staedion_dm.Leefbaarheid.LeefbaarheidsdossiersOpBuurt GROUP BY Jaar ORDER BY Jaar desc

(6 rows affected)

Total execution time: 00:00:00.020

Jaar,(No column name)
2021,90
2020,90
2019,82
2018,82
2017,83
2016,83


## 4. Aanvullen staedion\_dm.Leefbaarheid.TeamscoreOpBuurt + \[Leefbaarheid\].\[TeamscoreOpCluster\]

In [ ]:
/* 
-- ingelezen
SELECT * 
INTO [empire_staedion_data].[bik].[Teamscore2022]
FROM ff...ff
;
*/

In [9]:
SELECT Jaar,
       COUNT(*) AS AantalClusters,
       SUM(AantalWoningen) AS AantalWoningen,
       AVG([TeamscoreOverlast]) AS [TeamscoreOverlast],
       AVG([TeamscoreVervuiling]) AS [TeamscoreVervuiling],
       AVG([TeamscoreGemiddeldOverlastVervuiling]) AS [TeamscoreGemiddeldOverlastVervuiling],
       AVG([TeamscoreCriminaliteitWoonfraude]) AS [TeamscoreCriminaliteitWoonfraude],
       AVG([TeamscoreParticipatie]) AS [TeamscoreParticipatie],
       AVG([TeamscoreLeefkwaliteit]) AS [TeamscoreLeefkwaliteit],
       AVG([TeamscoreBeheerintensiteit]) AS [TeamscoreBeheerintensiteit],
       AVG([TeamscoreCijfer]) AS [TeamscoreCijfer]
FROM staedion_dm.Leefbaarheid.TeamscoreOpCluster
GROUP BY Jaar
ORDER BY Jaar desc
;

Warning: Null value is eliminated by an aggregate or other SET operation.

(4 rows affected)

Total execution time: 00:00:00.122

Jaar,AantalClusters,AantalWoningen,TeamscoreOverlast,TeamscoreVervuiling,TeamscoreGemiddeldOverlastVervuiling,TeamscoreCriminaliteitWoonfraude,TeamscoreParticipatie,TeamscoreLeefkwaliteit,TeamscoreBeheerintensiteit,TeamscoreCijfer
2021,678,36687,"6,81858407079646","6,912979351032448","6,865044247787611","7,615044247787611",NULL,"6,908554572271386","6,491150442477876","7,122861356932159"
2020,676,36346,"6,596153846153846","6,652366863905326","6,618343195266272","7,482248520710059","7,588757396449704","6,791420118343195","6,27810650887574","6,951775147928999"
2019,632,35247,"6,4414556962025316","6,504746835443038","6,473101265822785","7,419303797468355","7,6613924050632916","6,800632911392405","6,072784810126582","6,885443037974682"
2018,632,35162,NULL,NULL,"6,420886075949367","7,325949367088608","6,564873417721519","6,7136075949367084","6,2215189873417724","6,6493670886076"


In [ ]:
SELECT Jaar,
       COUNT(*) AS AantalBuurten,
       AVG([TeamscoreOverlast]) AS [TeamscoreOverlast],
       AVG([TeamscoreVervuiling]) AS [TeamscoreVervuiling],
       AVG([TeamscoreGemiddeldOverlastVervuiling]) AS [TeamscoreGemiddeldOverlastVervuiling],
       AVG([TeamscoreCriminaliteitWoonfraude]) AS [TeamscoreCriminaliteitWoonfraude],
       AVG([TeamscoreParticipatie]) AS [TeamscoreParticipatie],
       AVG([TeamscoreLeefkwaliteit]) AS [TeamscoreLeefkwaliteit],
       AVG([TeamscoreBeheerintensiteit]) AS [TeamscoreBeheerintensiteit],
       AVG([TeamscoreCijfer]) AS [TeamscoreCijfer]
FROM staedion_dm.Leefbaarheid.TeamscoreOpBuurt
GROUP BY Jaar
ORDER BY Jaar desc;
